In [1]:
from google.colab import drive
drive.mount('/content/drive')
# !pip install accelerate -U
!pip install evaluate
!pip install datasets
!pip install pymorphy2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import nltk
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import re
from datasets import Dataset, DatasetDict
import datasets
from transformers import Trainer, TrainingArguments, set_seed
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from transformers import DataCollatorWithPadding
import evaluate
from transformers import EvalPrediction
import os
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
import pymorphy2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Подготовка данных

In [ ]:
# считываем данные из ноутбука "X5 tematic model Логика + k-means"
data = pd.read_csv('/content/drive/MyDrive/cleaned_predicted_class_rus.csv')
# считываем выбросы из ноутбука "X5 tematic model Логика + k-means"
outliers = pd.read_csv('/content/drive/MyDrive/outliers_RUS.csv')
# считываем логически размеченные данные из ноутбука "X5 tematic model Логика + k-means"
df_logic = pd.read_csv('/content/drive/MyDrive/df_topics_logic.csv')

In [ ]:
# функция которая помогает почистить классы после kmeans
def classifactions(data):
  if data == 'Проблема с получением':
    return 'Проблема с получением'.lower()
  elif data == 'Соедините с оператором':
    return 'уточнить вопрос'.lower()
  elif data == 'Получатель подтверждает что заберт заказ':
    return 'уточнить вопрос'.lower()
  elif data == 'Срок доставки':
    return 'узнать статус посылки'.lower()
  elif data == 'Посылка не пришла':
    return 'Посылка не пришла'.lower()
  elif data == 'На кассе':
    return 'На кассе'.lower()
  elif data == 'Получить информацию о заказе':
    return 'узнать статус посылки'.lower()
  elif data == 'Отказаться от заказа':
    return 'уточнить вопрос'.lower()
  elif data == 'Проблема на пункте выдачи':
    return 'Проблема на пункте выдачи'.lower()
  elif data == 'Местонахождение заказа':
    return 'узнать статус посылки'.lower()
  elif data == 'Узнать дату получения':
    return 'Узнать дату получения или адрес доставки'.lower()
  elif data == 'Заказ пришл':
    return 'Посылка пришла не на тот адрес или не дошла'.lower()
  elif data == 'Не работает постамат':
    return 'не открывается ячейка или не работает постамат'.lower()
  elif data == 'Проблемы с содержимым':
    return 'брак возврат или повреждение'.lower()
  elif data == 'Получить информацию о доставке':
    return 'узнать статус посылки'.lower()
  elif data == 'Узнать срок хранения':
    return 'Узнать срок хранения истек срок хранения'.lower()
  elif data == 'Заказ пришл не по тому адресу':
    return 'Заказ пришел но возникли проблемы с получением'.lower()
  elif data == 'Проблема с открытием ячейки':
    return 'не открывается ячейка или не работает постамат'.lower()
  elif data == 'Номер заказа':
    return 'узнать статус посылки'.lower()
  elif data == 'Как забрать заказ':
    return 'Проблема с получением'.lower()
  elif data == 'Как забрать заказ':
    return 'Проблема с получением'.lower()
  elif data == 'Мне нужно взять посылку из постамата':
    return 'Проблема с получением'.lower()
  elif data == 'СМС для получения заказа':
    return 'уточнить вопрос'.lower()
  else:
    return 'уточнить вопрос'.lower()

data['topic'] = data.predicted_class.apply(lambda x: classifactions(x))
outliers['topic'] = 'связать с оператором'

In [ ]:
# объединяем логику, kmeans и выбросы
df = pd.concat([df_logic[['text','topic']], data[['text','topic']]], ignore_index=True)
df = pd.concat([df[['text','topic']], outliers[['text','topic']]], ignore_index=True)
df.head()

,text,topic
0,потеряли код получения заказа,узнать код открытия ячейки
1,мне не пришел код для получения посылки,узнать код открытия ячейки
2,связаться с оператором,связать с оператором
3,я хочу узнать код получения посылки,узнать код открытия ячейки
4,оператора можно связаться,связать с оператором


In [ ]:
# частота классов в данных
df.topic.value_counts()

topic
узнать код открытия ячейки                        391234
связать с оператором                              367547
узнать статус посылки                             264543
проблема с получением                             117750
не открывается ячейка или не работает постамат    115142
уточнить вопрос                                    75070
продлить срок хранения заказа                      25015
посылка не пришла                                  10576
вернуть заказ                                       9414
на кассе                                            9360
заказ потерян                                       8422
инвентаризация                                      7834
изменить адрес доставки заказа                      7742
проблема на пункте выдачи                           6851
узнать дату получения или адрес доставки            4342
посылка пришла не на тот адрес или не дошла         3657
изъятие посылки                                     3306
брак возврат или поврежде

In [ ]:
# словарь с классами
tp = df.topic.unique()

In [ ]:
# переводим классы в формат чисел
def classifactions(data):
  if data == tp[0]:
    return 1
  elif data == tp[1]:
    return 2
  elif data == tp[2]:
    return 3
  elif data == tp[3]:
    return 4
  elif data == tp[4]:
    return 5
  elif data == tp[5]:
    return 6
  elif data == tp[6]:
    return 7
  elif data == tp[7]:
    return 8
  elif data == tp[8]:
    return 9
  elif data == tp[9]:
    return 10
  elif data == tp[10]:
    return 11
  elif data == tp[11]:
    return 12
  elif data == tp[12]:
    return 13
  elif data == tp[13]:
    return 14
  elif data == tp[14]:
    return 15
  elif data == tp[15]:
    return 16
  elif data == tp[16]:
    return 17
  elif data == tp[17]:
    return 18
  elif data == tp[18]:
    return 19
  elif data == tp[19]:
    return 20

df['topic_num'] = df.topic.apply(lambda x: classifactions(x))

In [ ]:
df.head()

,text,topic,topic_num
0,потеряли код получения заказа,узнать код открытия ячейки,1
1,мне не пришел код для получения посылки,узнать код открытия ячейки,1
2,связаться с оператором,связать с оператором,2
3,я хочу узнать код получения посылки,узнать код открытия ячейки,1
4,оператора можно связаться,связать с оператором,2


In [ ]:
# делим данные на обучающую и тестовую выборки
# тест будем использовать для проверки модели на отложенной выборке при необходимости
train, test = train_test_split(df, test_size=0.3)

In [ ]:
# удаляем лишние колонки
train = train.drop('topic', axis=1)
test = test.drop('topic', axis=1)

In [ ]:
# очистка текстовых данных
def preprocess_text(data, stopwords=stopwords.words('russian')):
  """ Очистка текста и удаление стоп слов """
  text = re.sub('ё', 'е', data[0])
  text = ''.join([char for char in text if char not in string.punctuation])  # Удаление знаков препинания
  text = ' '.join(text.split())  # Удаление повторяющихся пробелов
  text = text.strip()
  # удаление стоп слов
  text = [w for w in text.split() if w.lower() not in stopwords]
  # удаление слов короче 3 символов
  text  = [w for w in text if len(w) >=3]
  return ' '.join(text)

# нормализация текстовых данных
morph = pymorphy2.MorphAnalyzer()
def lemmatization_text(data, morph=morph):
  """ Приведение слов к нормальной форме """
  words = data.split()
  normalized_words = [morph.parse(word)[0].normal_form for word in words]
  return ' '.join(normalized_words)


In [ ]:
# чистим обучающую и тестовую выборки
train['text_clean'] = train['text'].apply(preprocess_text)
train['text_clean'] = train['text'].apply(lemmatization_text)

test['text_clean'] = test['text'].apply(preprocess_text)
test['text_clean'] = test['text'].apply(lemmatization_text)


In [ ]:
# удаляем не нужные столбцы
train = train.drop('text', axis=1)
test = test.drop('text', axis=1)
# преименовываем колонки в удобное название
train.rename(columns={'text_clean':'text'}, inplace=True)
test.rename(columns={'text_clean':'text'}, inplace=True)

train.rename(columns={'topic_num':'label'}, inplace=True)
test.rename(columns={'topic_num':'label'}, inplace=True)

In [ ]:
# делим на тренировчный и валидационный датасет
df_train, df_valid= train_test_split(train[['text', 'label']], test_size=0.2)
# df_train.to_csv('/content/drive/MyDrive/df_train_roberta.csv')
# df_valid.to_csv('/content/drive/MyDrive/df_valid_roberta.csv')

In [3]:
# df_train = pd.read_csv('/content/drive/MyDrive/df_train_roberta.csv')
# df_valid = pd.read_csv('/content/drive/MyDrive/df_valid_roberta.csv')

In [5]:
# создаем словарь для подачи в модель
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop = True)),
    "valid": Dataset.from_pandas(df_valid.reset_index(drop = True))
    })
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 803734
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 200934
    })
})

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
# токенизируем текстовые данные
def preprocess_data(examples):

    encoding = tokenizer(examples['text'])
    return encoding

# используем эмбэндинги DeepPavlov/rubert-base-cased
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = True)

encoded_dataset = dataset.map(preprocess_data, batched = True)
encoded_dataset = encoded_dataset.remove_columns(['text'])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch")
encoded_dataset

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/803734 [00:00<?, ? examples/s]

Map:   0%|          | 0/200934 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 803734
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200934
    })
})

## Обучение модели

In [9]:
# определяем метрики качества
ACCURACY = evaluate.load("accuracy")
MCC = evaluate.load("matthews_correlation")

# функция для расчета метрик
def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    preds = np.argmax(preds, axis=1)

    acc_result = ACCURACY.compute(predictions=preds, references=p.label_ids)
    mcc_result = MCC.compute(predictions=preds, references=p.label_ids)
    result = {"accuracy": acc_result["accuracy"], "mcc": mcc_result["matthews_correlation"]}

    return result


metric = evaluate.load("accuracy")

In [ ]:
# загружаем модель
model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels = 21)

training_args = TrainingArguments(
    output_dir=f"Roberta",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.1,
    num_train_epochs=3,
    warmup_ratio=0.1,
    optim="adamw_torch",
    save_strategy="no",
    save_total_limit=1,
    group_by_length=True,
    report_to="none"

)

# обучаем модель
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['valid'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Mcc
1,0.141800,0.142565,0.955508,0.945039


Epoch,Training Loss,Validation Loss,Accuracy,Mcc
1,0.141800,0.142565,0.955508,0.945039
